# Test api openF1  
  
Le but de ce notebook est de tester les différents appels avant de les industrialiser

In [15]:
import requests
import json
from genericpath import exists
import os
import pandas as pd

In [33]:
# Url de base de l'api 
url_base = "https://api.openf1.org/v1/"

# A GET request to the API
response = requests.get(url_base+"pit?date>2025-03-01&date<2025-04-01")

# Print the response
display(pd.json_normalize(response.json()))

,date,session_key,stop_duration,lap_number,lane_duration,driver_number,meeting_key,pit_duration
0,2025-03-14T01:30:03.187000+00:00,9686,NaN,1,NaN,27,1254,NaN
1,2025-03-14T01:30:14.530000+00:00,9686,NaN,1,NaN,6,1254,NaN
2,2025-03-14T01:30:15.871000+00:00,9686,NaN,1,NaN,4,1254,NaN
3,2025-03-14T01:30:19.204000+00:00,9686,NaN,1,NaN,5,1254,NaN
4,2025-03-14T01:30:28.407000+00:00,9686,NaN,1,NaN,10,1254,NaN
...,...,...,...,...,...,...,...,...
703,2025-03-23T07:58:46.021000+00:00,9998,2.5,33,22.234,6,1255,22.234
704,2025-03-23T08:01:56.370000+00:00,9998,2.3,35,22.583,22,1255,22.583
705,2025-03-23T08:03:32.324000+00:00,9998,3.2,36,22.995,18,1255,22.995
706,2025-03-23T08:04:48.706000+00:00,9998,2.3,37,22.364,44,1255,22.364


Enregistrement avec périodisation sur l'année pour les tables que nous pouvons

In [12]:
year=["2023","2024","2025","2026"]
nom="meetings"

if not exists(f"/workspaces/F1_website/data/raw/{nom}"):
        os.mkdir(f"/workspaces/F1_website/data/raw/{nom}")

for y in year:
    response = requests.get(url_base+nom+"?year="+y)

    if not exists(f"/workspaces/F1_website/data/raw/{nom}/{y}/"):
        os.mkdir(f"/workspaces/F1_website/data/raw/{nom}/{y}/")
    with open(f"/workspaces/F1_website/data/raw/{nom}/{y}/{nom}_{y}.json","w") as fichier:
        json.dump(response.json(),fichier)

Test de lecture

In [28]:
path_table=os.walk(f"/workspaces/F1_website/data/raw/{nom}")

df=pd.DataFrame()
for fichiers in path_table:
    for fichier in fichiers[2]:
        with open(fichiers[0]+"/"+fichier) as f:
            data = json.load(f)
            df=pd.concat([df,pd.json_normalize(data)])

display(df)

,meeting_key,meeting_name,meeting_official_name,location,country_key,country_code,country_name,country_flag,circuit_key,circuit_short_name,circuit_type,circuit_image,gmt_offset,date_start,date_end,year
0,1228,Pre-Season Testing,FORMULA 1 ARAMCO PRE-SEASON TESTING 2024,Sakhir,36,BRN,Bahrain,https://media.formula1.com/content/dam/fom-web...,63,Sakhir,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-02-21T07:00:00+00:00,2024-02-23T16:00:00+00:00,2024
1,1229,Bahrain Grand Prix,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Sakhir,36,BRN,Bahrain,https://media.formula1.com/content/dam/fom-web...,63,Sakhir,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-02-29T11:30:00+00:00,2024-03-02T17:00:00+00:00,2024
2,1230,Saudi Arabian Grand Prix,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,Jeddah,153,KSA,Saudi Arabia,https://media.formula1.com/content/dam/fom-web...,149,Jeddah,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,03:00:00,2024-03-07T13:30:00+00:00,2024-03-09T19:00:00+00:00,2024
3,1231,Australian Grand Prix,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,Melbourne,5,AUS,Australia,https://media.formula1.com/content/dam/fom-web...,10,Melbourne,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,11:00:00,2024-03-22T01:30:00+00:00,2024-03-24T06:00:00+00:00,2024
4,1232,Japanese Grand Prix,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,Suzuka,4,JPN,Japan,https://media.formula1.com/content/dam/fom-web...,46,Suzuka,Permanent,https://media.formula1.com/content/dam/fom-web...,09:00:00,2024-04-05T02:30:00+00:00,2024-04-07T07:00:00+00:00,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,1272,Mexico City Grand Prix,FORMULA 1 GRAN PREMIO DE LA CIUDAD DE MÉXICO 2025,Mexico City,8,MEX,Mexico,https://media.formula1.com/content/dam/fom-web...,65,Mexico City,Permanent,https://media.formula1.com/content/dam/fom-web...,-06:00:00,2025-10-24T18:30:00+00:00,2025-10-26T22:00:00+00:00,2025
21,1273,São Paulo Grand Prix,FORMULA 1 MSC CRUISES GRANDE PRÊMIO DE SÃO PAU...,São Paulo,10,BRA,Brazil,https://media.formula1.com/content/dam/fom-web...,14,Interlagos,Permanent,https://media.formula1.com/content/dam/fom-web...,-03:00:00,2025-11-07T14:30:00+00:00,2025-11-09T19:00:00+00:00,2025
22,1274,Las Vegas Grand Prix,FORMULA 1 HEINEKEN LAS VEGAS GRAND PRIX 2025,Las Vegas,19,USA,United States,https://media.formula1.com/content/dam/fom-web...,152,Las Vegas,Temporary - Street,https://media.formula1.com/content/dam/fom-web...,-08:00:00,2025-11-21T00:30:00+00:00,2025-11-23T06:00:00+00:00,2025
23,1275,Qatar Grand Prix,FORMULA 1 QATAR AIRWAYS QATAR GRAND PRIX 2025,Lusail,149,QAT,Qatar,https://media.formula1.com/content/dam/fom-web...,150,Lusail,Permanent,https://media.formula1.com/content/dam/fom-web...,03:00:00,2025-11-28T13:30:00+00:00,2025-11-30T18:00:00+00:00,2025


In [22]:
kretourne=os.walk(f"/workspaces/F1_website/data/raw/{nom}")

for kelement in kretourne :
    print ( kelement [ 0 ] ) # affiche le chemin absolue de l'élément exploré
    print ( " - ses répertoires :" )
    for krepertoire in kelement [ 1 ] : # pour tous les sous répertoires dans répertoire kelement [ 0 ] 
        print ( " -- " , krepertoire ) # affiche le nom d'un sous répertoire qui sera exploré ensuite
    print ( " - ses fichiers :"  )
    for kfichier in kelement [ 2 ] : # pour tous les fichiers dans répertoire kelement [ 0 ] 
        print ( " -- " , kfichier ) # affiche le nom d'un fichier
    print ( )

/workspaces/F1_website/data/raw/meetings
 - ses répertoires :
 --  2024
 --  2023
 --  2026
 --  2025
 - ses fichiers :

/workspaces/F1_website/data/raw/meetings/2024
 - ses répertoires :
 - ses fichiers :
 --  meetings_2024.json

/workspaces/F1_website/data/raw/meetings/2023
 - ses répertoires :
 - ses fichiers :
 --  meetings_2023.json

/workspaces/F1_website/data/raw/meetings/2026
 - ses répertoires :
 - ses fichiers :
 --  meetings_2026.json

/workspaces/F1_website/data/raw/meetings/2025
 - ses répertoires :
 - ses fichiers :
 --  meetings_2025.json

